<a href="https://colab.research.google.com/github/Ryong1998/snowball/blob/main/4_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://pmorissette.github.io/bt/

# 백테스트팅

- 미래예측을 하는 모델을 생성하기에는 금리와 시장 상황에 따라 수익률이 천차만별
- 해당 기간동안의 해당 전략으로 ETF를 선별했을 때 어떤 수익률이 나왔는지 확인하는 과정 진행

## 데이터 불러오기

In [1]:
import pandas as pd
# 출력할 최대 컬럼 수를 설정 (예: 20개로 설정)
pd.set_option('display.max_columns', 30)
df_etf_evaluate = pd.read_pickle('/content/drive/MyDrive/snowball/data/After_data_preprocessing/df_etf_evaluate.pkl')
print(df_etf_evaluate.info())
df_etf_evaluate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228229 entries, 0 to 228228
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   etf_code                 228229 non-null  int64         
 1   etf_date                 228229 non-null  datetime64[ns]
 2   etf_real_price           228229 non-null  float64       
 3   etf_price                228229 non-null  int64         
 4   etf_revenue              228229 non-null  float64       
 5   etf_operatingincomeloss  228229 non-null  float64       
 6   etf_profitloss           228229 non-null  float64       
 7   etf_assets               228229 non-null  float64       
 8   etf_liabilities          228229 non-null  float64       
 9   etf_equity               228229 non-null  float64       
 10  etf_per                  228229 non-null  float64       
 11  etf_pbr                  228229 non-null  float64       
 12  etf_psr         

,etf_code,etf_date,etf_real_price,etf_price,etf_revenue,etf_operatingincomeloss,etf_profitloss,etf_assets,etf_liabilities,etf_equity,etf_per,etf_pbr,etf_psr,etf_operating_ratio,etf_profit_ratio,etf_debt_ratio,etf_roe
0,69500,2017-01-02,24383.71174,26415,38271.918037,3019.687683,2176.379620,89955.773639,63346.245309,26609.528330,12.137129,0.992690,0.690193,7.890087,5.686623,238.058505,8.178949
1,69500,2017-01-03,24593.71174,26625,38271.918037,3019.687683,2176.379620,89955.773639,63346.245309,26609.528330,12.233619,1.000581,0.695680,7.890087,5.686623,238.058505,8.178949
2,69500,2017-01-04,24623.71174,26655,38271.918037,3019.687683,2176.379620,89955.773639,63346.245309,26609.528330,12.247404,1.001709,0.696464,7.890087,5.686623,238.058505,8.178949
3,69500,2017-01-05,24518.71174,26550,38271.918037,3019.687683,2176.379620,89955.773639,63346.245309,26609.528330,12.199159,0.997763,0.693720,7.890087,5.686623,238.058505,8.178949
4,69500,2017-01-06,24633.71174,26665,38271.918037,3019.687683,2176.379620,89955.773639,63346.245309,26609.528330,12.251999,1.002085,0.696725,7.890087,5.686623,238.058505,8.178949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228224,448570,2022-12-23,9339.12918,9645,41237.154403,3062.602751,2214.324945,78988.223848,58725.799013,20262.424834,4.355729,0.476004,0.233891,7.426804,5.369733,289.826117,10.928233
228225,448570,2022-12-26,9324.12918,9630,41237.154403,3062.602751,2214.324945,78988.223848,58725.799013,20262.424834,4.348955,0.475264,0.233527,7.426804,5.369733,289.826117,10.928233
228226,448570,2022-12-27,9384.12918,9690,41237.154403,3062.602751,2214.324945,78988.223848,58725.799013,20262.424834,4.376052,0.478225,0.234982,7.426804,5.369733,289.826117,10.928233
228227,448570,2022-12-28,9384.12918,9690,41237.154403,3062.602751,2214.324945,78988.223848,58725.799013,20262.424834,4.376052,0.478225,0.234982,7.426804,5.369733,289.826117,10.928233


## 조회할 날짜 계산하는 함수 생성

In [2]:
etf_date_list = list(set(df_etf_evaluate['etf_date']))
etf_date_list = sorted(etf_date_list)
etf_date_list[:3]

[Timestamp('2017-01-02 00:00:00'),
 Timestamp('2017-01-03 00:00:00'),
 Timestamp('2017-01-04 00:00:00')]

In [3]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 몇 개월 단위로 값들을 추가할
def get_specific_date(duration):
    date_list = []

    # 시작 날짜와 끝 날짜 설정
    start_date = datetime(2017, 1, 2)
    end_date = datetime(2022, 12, 29)

    date_list.append(start_date)

    current_date = start_date

    while current_date <= end_date:

      current_date += relativedelta(months=duration)
      next_date = current_date

      if current_date in etf_date_list:
          date_list.append(current_date)

      else:
        while (next_date not in date_list) and (next_date<datetime(2022, 12, 29)):
          next_date += timedelta(days=1)
          if next_date in etf_date_list:
              date_list.append(next_date)
              break

    date_list.append(end_date)

    return date_list



specific_date_list = get_specific_date(6)
specific_date_list


[datetime.datetime(2017, 1, 2, 0, 0),
 datetime.datetime(2017, 7, 3, 0, 0),
 datetime.datetime(2018, 1, 2, 0, 0),
 datetime.datetime(2018, 7, 2, 0, 0),
 datetime.datetime(2019, 1, 2, 0, 0),
 datetime.datetime(2019, 7, 2, 0, 0),
 datetime.datetime(2020, 1, 2, 0, 0),
 datetime.datetime(2020, 7, 2, 0, 0),
 datetime.datetime(2021, 1, 4, 0, 0),
 datetime.datetime(2021, 7, 2, 0, 0),
 datetime.datetime(2022, 1, 3, 0, 0),
 datetime.datetime(2022, 7, 4, 0, 0),
 datetime.datetime(2022, 12, 29, 0, 0)]

## KODEX 200 데이터프레임 생성

In [4]:
df_kodex_200 = df_etf_evaluate.loc[(df_etf_evaluate['etf_code']==69500),['etf_date','etf_price']].copy()
df_kodex_200.head()

,etf_date,etf_price
0,2017-01-02,26415
1,2017-01-03,26625
2,2017-01-04,26655
3,2017-01-05,26550
4,2017-01-06,26665


In [5]:
df_kodex_200['return_rate'] = round((100*(df_kodex_200['etf_price'] - 26415))/26415, 2)
df_kodex_200

,etf_date,etf_price,return_rate
0,2017-01-02,26415,0.00
1,2017-01-03,26625,0.80
2,2017-01-04,26655,0.91
3,2017-01-05,26550,0.51
4,2017-01-06,26665,0.95
...,...,...,...
1470,2022-12-23,30235,14.46
1471,2022-12-26,30280,14.63
1472,2022-12-27,30495,15.45
1473,2022-12-28,30135,14.08


## 수익률 그래프

In [6]:
df_etf_evaluate.loc[((df_etf_evaluate['etf_per']>=5)&(df_etf_evaluate['etf_per']<=8)&(df_etf_evaluate['etf_date']=='2017-01-02')),['etf_code','etf_price','etf_per']].sort_values('etf_per',ascending=False)

,etf_code,etf_price,etf_per
5332,91180,17600,7.986440
34849,139250,12305,7.982803
100258,252650,9890,7.859806
28949,138540,18485,7.640226
68002,223190,7680,5.120652


In [7]:
etf_info = df_etf_evaluate.loc[((df_etf_evaluate['etf_per']>=5)&(df_etf_evaluate['etf_per']<=8)&(df_etf_evaluate['etf_date']=='2017-01-02')),['etf_code','etf_price','etf_per']].sort_values('etf_per').iloc[0]
etf_info

etf_code     223190.000000
etf_price      7680.000000
etf_per           5.120652
Name: 68002, dtype: float64

In [8]:
etf_info['etf_code']
etf_info['etf_price']

7680.0

#### 평가지표 따른 etf 정보 추출 함수

In [56]:
# 평가지표에 맞는 코드와 가격을 조회하는 함수 생성


def get_etf_code_price(strategy, min_num, max_num, date, asc): # asc True이면 값이 낮은것들이 나옴, False 이면 높은것들이 나옴
  if asc == True:
    try:
      etf_info = df_etf_evaluate.loc[((df_etf_evaluate[strategy]>min_num)&(df_etf_evaluate[strategy]<=max_num)&(df_etf_evaluate['etf_date']==date)),['etf_code','etf_price',strategy]].sort_values(strategy, ascending=True).iloc[0]
      return etf_info['etf_code'], etf_info['etf_price'], etf_info[strategy]
    except:
      return 'error'
  else:
    try:
      etf_info = df_etf_evaluate.loc[((df_etf_evaluate[strategy]>min_num)&(df_etf_evaluate[strategy]<=max_num)&(df_etf_evaluate['etf_date']==date)),['etf_code','etf_price',strategy]].sort_values(strategy, ascending=False).iloc[0]
      return etf_info['etf_code'], etf_info['etf_price'], etf_info[strategy]
    except:
      return 'error'




print(get_etf_code_price('etf_per', 5, 8, '2017-01-02', True))

(223190.0, 7680.0, 5.120651977979314)


### 수익률 계산 함수

In [57]:
import math
def get_return_rate(strategy, min_num, max_num, duration, asc):

  specific_date_list = get_specific_date(duration)

  etf_code = 0

  before_etf_price = 0
  current_etf_price = 0

  accumulated_return_rate =0
  return_rate =0
  return_rate_dict = dict()

  etf_code_dict = dict()
  strategy_score_dict = dict()

  for i,date in enumerate(etf_date_list): # 모든 날짜들에 대해서 진행
    if date in specific_date_list: # 상품조회 날짜인 경우
      if i ==0:
        if get_etf_code_price(strategy, min_num, max_num, date, asc) == 'error':
          return 'error', 'error','error'
        etf_code, before_etf_price, strategy_score = get_etf_code_price(strategy, min_num, max_num, date, asc) # 가장 첫번째 날짜는 금융상품과 가격 설정

        return_rate_dict[date] =  return_rate
        etf_code_dict[date] = etf_code
        strategy_score_dict[date] = strategy_score

      else:
        if get_etf_code_price(strategy, min_num, max_num, date, asc) == 'error':
          return 'error', 'error','error'

        current_etf_price = df_etf_evaluate.loc[(df_etf_evaluate['etf_code']==etf_code)&(df_etf_evaluate['etf_date']==date),'etf_price'].iloc[0] # 조회 가격 불러오기
        return_rate = (100*(current_etf_price - before_etf_price))/before_etf_price # 수익률 조회
        return_rate = accumulated_return_rate + return_rate

        return_rate = round(return_rate,2)
        return_rate_dict[date] =  return_rate

        accumulated_return_rate = return_rate

        etf_code, before_etf_price, strategy_score = get_etf_code_price(strategy, min_num, max_num, date, asc) # 금융상품코드와 가격 갱신
        etf_code_dict[date] = etf_code
        strategy_score_dict[date] = strategy_score

    else:
      current_etf_price = df_etf_evaluate.loc[(df_etf_evaluate['etf_code']==etf_code)&(df_etf_evaluate['etf_date']==date),'etf_price'].iloc[0]
      return_rate = (100*(current_etf_price - before_etf_price))/before_etf_price

      return_rate = accumulated_return_rate + return_rate
      return_rate = round(return_rate,2)
      return_rate_dict[date] =  return_rate


  return return_rate_dict, etf_code_dict, strategy_score_dict

return_rate_dict, etf_code_dict, strategy_score_dict = get_return_rate('etf_per', 0, 20, 3, True)
if return_rate_dict =='error':
  print('retry!! the range is error')
else:
  print(etf_code_dict)
  print(strategy_score_dict)

{Timestamp('2017-01-02 00:00:00'): 223190.0, Timestamp('2017-04-03 00:00:00'): 223190.0, Timestamp('2017-07-03 00:00:00'): 223190.0, Timestamp('2017-10-10 00:00:00'): 223190.0, Timestamp('2018-01-02 00:00:00'): 223190.0, Timestamp('2018-04-02 00:00:00'): 223190.0, Timestamp('2018-07-02 00:00:00'): 223190.0, Timestamp('2018-10-02 00:00:00'): 223190.0, Timestamp('2019-01-02 00:00:00'): 223190.0, Timestamp('2019-04-02 00:00:00'): 223190.0, Timestamp('2019-07-02 00:00:00'): 223190.0, Timestamp('2019-10-02 00:00:00'): 91170.0, Timestamp('2020-01-02 00:00:00'): 91220.0, Timestamp('2020-04-02 00:00:00'): 91220.0, Timestamp('2020-07-02 00:00:00'): 91220.0, Timestamp('2020-10-05 00:00:00'): 91220.0, Timestamp('2021-01-04 00:00:00'): 91220.0, Timestamp('2021-04-02 00:00:00'): 91220.0, Timestamp('2021-07-02 00:00:00'): 91220.0, Timestamp('2021-10-05 00:00:00'): 223190.0, Timestamp('2022-01-03 00:00:00'): 223190.0, Timestamp('2022-04-04 00:00:00'): 244670.0, Timestamp('2022-07-04 00:00:00'): 28502

In [34]:
etf_code_dict.keys()

dict_keys([Timestamp('2017-01-02 00:00:00'), Timestamp('2017-04-03 00:00:00'), Timestamp('2017-07-03 00:00:00'), Timestamp('2017-10-10 00:00:00'), Timestamp('2018-01-02 00:00:00'), Timestamp('2018-04-02 00:00:00'), Timestamp('2018-07-02 00:00:00'), Timestamp('2018-10-02 00:00:00'), Timestamp('2019-01-02 00:00:00'), Timestamp('2019-04-02 00:00:00'), Timestamp('2019-07-02 00:00:00'), Timestamp('2019-10-02 00:00:00'), Timestamp('2020-01-02 00:00:00'), Timestamp('2020-04-02 00:00:00'), Timestamp('2020-07-02 00:00:00'), Timestamp('2020-10-05 00:00:00'), Timestamp('2021-01-04 00:00:00'), Timestamp('2021-04-02 00:00:00'), Timestamp('2021-07-02 00:00:00'), Timestamp('2021-10-05 00:00:00'), Timestamp('2022-01-03 00:00:00'), Timestamp('2022-04-04 00:00:00'), Timestamp('2022-07-04 00:00:00'), Timestamp('2022-10-04 00:00:00'), Timestamp('2022-12-29 00:00:00')])

In [35]:
return_rate_dict

{Timestamp('2017-01-02 00:00:00'): 0,
 Timestamp('2017-01-03 00:00:00'): 1.11,
 Timestamp('2017-01-04 00:00:00'): 0.91,
 Timestamp('2017-01-05 00:00:00'): 0.39,
 Timestamp('2017-01-06 00:00:00'): 0.72,
 Timestamp('2017-01-09 00:00:00'): 1.37,
 Timestamp('2017-01-10 00:00:00'): 0.98,
 Timestamp('2017-01-11 00:00:00'): 2.08,
 Timestamp('2017-01-12 00:00:00'): 3.12,
 Timestamp('2017-01-13 00:00:00'): 2.6,
 Timestamp('2017-01-16 00:00:00'): 2.08,
 Timestamp('2017-01-17 00:00:00'): 2.47,
 Timestamp('2017-01-18 00:00:00'): 2.54,
 Timestamp('2017-01-19 00:00:00'): 2.67,
 Timestamp('2017-01-20 00:00:00'): 2.28,
 Timestamp('2017-01-23 00:00:00'): 2.28,
 Timestamp('2017-01-24 00:00:00'): 2.28,
 Timestamp('2017-01-25 00:00:00'): 2.54,
 Timestamp('2017-01-26 00:00:00'): 3.45,
 Timestamp('2017-01-31 00:00:00'): 2.28,
 Timestamp('2017-02-01 00:00:00'): 2.8,
 Timestamp('2017-02-02 00:00:00'): 2.34,
 Timestamp('2017-02-03 00:00:00'): 2.41,
 Timestamp('2017-02-06 00:00:00'): 2.47,
 Timestamp('2017-02-0

### 수익률 시각화 함수

In [89]:
import plotly.graph_objects as go

def show_graph(dictionary, strategy, min_num, max_num, duration):
  # 딕셔너리의 키와 값 추출
  keys = list(dictionary.keys())
  values = list(dictionary.values())

  # plotly의 line chart 생성
  fig = go.Figure(data=go.Scatter(x=df_kodex_200['etf_date'], y=df_kodex_200['return_rate'], mode='lines', name='Kodex 200'))



  # plotly의 line chart 생성
  fig.add_trace(go.Scatter(x=keys, y=values, mode='lines', name= strategy + ' '+str(min_num)+'~'+str(max_num)+' '+str(duration)+'m'))

  # 그래프 레이아웃 설정
  fig.update_layout(title= strategy+' '+str(min_num)+'~'+str(max_num)+' '+str(duration)+'m'+' 수익률 그래프',
                    xaxis_title='날짜',
                    yaxis_title='수익률 추이')


  # 그래프 표시
  fig.show()
  print('최종 수익률 :', dictionary[pd.Timestamp('2022-12-29 00:00:00')])

show_graph(return_rate_dict, 'etf_per', 0, 20, 3)

최종 수익률 : -36.41


## 영업이익률

### 수치 변경

In [90]:
range_list = [[13,16],[10,13],[7,10],[4,7],[-1,4]]
for range in range_list:
  input_data = ('etf_operating_ratio', range[0], range[1], 6)
  return_rate_dict, etf_code_dict, strategy_score_dict = get_return_rate(*input_data, False) # 영업이익률은 높은것이 좋음으로 False를 수용
  if return_rate_dict =='error':
    print('retry!! the range is error')
  else:
    show_graph(return_rate_dict, *input_data)
    print()

최종 수익률 : 100.98



최종 수익률 : 24.45



최종 수익률 : 4.46



최종 수익률 : 20.11



최종 수익률 : -36.41



### duration 변경

In [86]:
duration_list = [3,4,6,12]
for duration in duration_list:
  input_data = ('etf_operating_ratio', 13, 16, duration)
  return_rate_dict, etf_code_dict, strategy_score_dict = get_return_rate(*input_data, False) # 영업이익률은 높은것이 좋음으로 False를 수용
  if return_rate_dict =='error':
    print('retry!! the range is error')
  else:
    show_graph(return_rate_dict, *input_data)
    print()

최종 수익률 : 108.63


최종 수익률 : 89.71


최종 수익률 : 100.98


최종 수익률 : 101.89
